In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers
import tensorflow_datasets as tfds

# Load the 'horses_or_humans' dataset
train_data, test_data = tfds.load('horses_or_humans', split=['train', 'test'], as_supervised=True)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

def preprocess_img(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

train_data = train_data.map(preprocess_img)
test_data = test_data.map(preprocess_img)

# Convert datasets to numpy arrays
train_images = np.array([image.numpy() for image, _ in train_data])
train_labels = np.array([label.numpy() for _, label in train_data])
test_images = np.array([image.numpy() for image, _ in test_data])
test_labels = np.array([label.numpy() for _, label in test_data])

# Define the batch size
batch_size = 32

# Create the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False
    
# Create a new model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_datagen.flow(train_images, train_labels, batch_size=batch_size),
    steps_per_epoch=len(train_images) // batch_size,
    epochs=10,
    validation_data=(test_images, test_labels),
    validation_steps=len(test_images) // batch_size
)

# Evaluate the model on the test set
test_score = model.evaluate(test_images, test_labels, batch_size=batch_size)
print(f'Test loss: {test_score[0]}, Test accuracy: {test_score[1]}')


ModuleNotFoundError: No module named 'tensorflow_datasets'

In [12]:
# Select a sample from the dataset for prediction
sample_image, sample_label = next(iter(test_data))
sample_image = sample_image.numpy().reshape((1, 224, 224, 3))

# Predict the output
prediction = model.predict(sample_image)

print("Predicted Output:", prediction)
print("Actual Label:", sample_label.numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
Predicted Output: [[6.116732e-37]]
Actual Label: 0


In [7]:
pip install tensorflow-datasets


  Obtaining dependency information for tensorflow-datasets from https://files.pythonhosted.org/packages/fe/18/4865973f5469cfe33bbe1cfc2f1918335eb44f4cc3d316c1bce22c1af2bc/tensorflow_datasets-4.9.4-py3-none-any.whl.metadata
  Using cached tensorflow_datasets-4.9.4-py3-none-any.whl.metadata (9.2 kB)
  Obtaining dependency information for etils[enp,epath,etree]>=0.9.0 from https://files.pythonhosted.org/packages/d0/3c/784b5f94bcad62a4167f2347fe1095b627433c22a54aecc4504237494b81/etils-1.8.0-py3-none-any.whl.metadata
  Using cached etils-1.8.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached promise-2.3-py3-none-any.whl
  Obtaining dependency information for tensorflow-metadata from https://files.pythonhosted.org/packages/41/23/3705c7139886c079ef4c0e3be56a5a1fb90e9ee413a4b7caaee0ee0ea6fe/tensorflow_metadata-1.14.0-py3-none-any.whl.metadata
  Using cached tensorflow_metadata-1.14.0-py3-none-any.whl.metadata (2.1 kB)
  Obtaining dependency information for importlib_resources from https://fil